In [2]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

from ase.io import read, write

pristine_slab = read("GaN_0001_3x3_pristine_slab.cif")
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
# AdsorbateSiteFinder(slab, selective_dynamics=False, height=0.9, mi_vec=None)
sites = site_finder.find_adsorption_sites(put_inside=False, symm_reduce=False)
# find_adsorption_sites(distance=2.0, put_inside=True, symm_reduce=0.01, near_reduce=0.01, positions=['ontop', 'bridge', 'hollow'], no_obtuse_hollow=True)

In [3]:
for type in sites.keys():
    print(f"{type} sites has total {len(sites[type])}")

ontop sites has total 18
bridge sites has total 69
hollow sites has total 8
all sites has total 95


In [4]:
import numpy as np
ads_positions = sites['all']
print(f"adsorption coordinates are")
print(ads_positions)
# take just the first 18 sites after inspection
# select_positions = ads_positions[:18]

# take all positions
select_positions = ads_positions

adsorption coordinates are
[array([ 0.        ,  0.        , 15.26569384]), array([12.86517608,  7.42771287, 14.62006789]), array([ 3.21630608,  5.57080555, 15.26569384]), array([11.25704716,  4.64235188, 14.62006789]), array([ 1.60812892,  2.78536099, 15.26569384]), array([ 9.64887   ,  1.85690733, 14.62006789]), array([ 6.43261216,  0.        , 15.26569384]), array([ 9.64891824,  7.42771287, 14.62006789]), array([ 9.64891824,  5.57080555, 15.26569384]), array([ 8.04078933,  4.64235188, 14.62006789]), array([ 8.04074108,  2.78536099, 15.26569384]), array([ 6.43261216,  1.85690733, 14.62006789]), array([ 3.21625784,  0.        , 15.26569384]), array([ 6.43256392,  7.42771287, 14.62006789]), array([ 6.43256392,  5.57080555, 15.26569384]), array([ 4.824435  ,  4.64235188, 14.62006789]), array([ 4.82438676,  2.78536099, 15.26569384]), array([ 3.21625784,  1.85690733, 14.62006789]), array([32.96702879, 14.39119892, 14.62006789]), array([32.96702879, 13.46274525, 14.94288087]), array([33.77

In [5]:
# adsorb onto the sites
from pymatgen.core.structure import Molecule
ads_mol = Molecule(['Ga'], [[0,0,0]])
for ads_coord in select_positions:
    slab = site_finder.add_adsorbate(ads_mol, ads_coord, repeat=None, translate=False, reorient=False)
    site_finder.slab = slab

In [6]:
len(site_finder.slab)

131

In [7]:
from pymatgen.io.cif import CifWriter
# write out slab
cif_slab = CifWriter(site_finder.slab)
cif_slab.write_file("pymatgen_GaN_0001_ads_Ga_all_adsorbed_slab.cif")

Manually check the file, looks legit

In [8]:
import os
import sys
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/sgmc_surf")

# do a sample run
from mcmc import mcmc_run

import ase
from ase.calculators.lammpsrun import LAMMPS
from ase.io import read

import numpy as np

from htvs.djangochem.pgmols.utils import surfaces

In [9]:
all_adsorbed = read("pymatgen_GaN_0001_ads_Ga_all_adsorbed_slab.cif")
print(f"all adsorbed {len(all_adsorbed)}")

# Get pristine surface
# GaN 0001 surface
atoms = read('GaN_hexagonal.cif')

# supercell_atoms = atoms*(2,2,2)
# supercell_atoms.write('GaN_hexagonal_2x2.cif')

supercell_atoms = atoms*(3,3,3)
supercell_atoms.write('GaN_hexagonal_3x3.cif')

slab, surface_atoms = surfaces.surface_from_bulk(supercell_atoms, [0,0,0,-1], size=[3,3], vacuum=10)
print(f"pristine slab {len(slab)}")

all adsorbed 131
pristine slab 36


/home/dux/miniconda3/envs/mlenv/lib/python3.8/site-packages/catkit/gen/surface.py:334: UserWarning: Your slab has no bulk atoms and may be too thin to identify surface atoms correctly. This may cause surface adsorption site identification to fail.
  warnings.warn(


In [17]:
select_positions = all_adsorbed.get_positions()[len(slab):]

In [18]:
num_ads_atoms = 12 # needs to have so many atoms

# try 2003 tersoff potential 
parameters = {
    'pair_style': 'tersoff',
    'pair_coeff': ['* * GaN.tersoff Ga N']
}
potential_file = os.path.join(os.environ["LAMMPS_POTENTIALS"], 'GaN.tersoff')
lammps_calc = LAMMPS(files=[potential_file], keep_tmp_files=False, keep_alive=False, tmp_dir="/home/dux/surface_sampling/tmp_files")
lammps_calc.set(**parameters)

element = 'Ga'
ads = ase.Atoms(element)

# canonical with relaxation
num_sweeps = 1
surface_name = "GaN_0001_3x3"
alpha = 0.99
slab, surface_atoms = surfaces.surface_from_bulk(supercell_atoms, [0,0,0,-1], size=[3,3], vacuum=10)
# set surface atoms from the other side
all_atoms = np.arange(len(slab))
curr_surf_atoms = slab.get_surface_atoms()
new_surf_atoms = np.setdiff1d(all_atoms, curr_surf_atoms)
slab.set_surface_atoms(new_surf_atoms)
# invert the positions
slab.set_scaled_positions(1 - slab.get_scaled_positions())

# try positive chem pot
chem_pot = 5
history, energy_hist, frac_accept_hist, adsorption_count_hist = mcmc_run(num_sweeps=num_sweeps, temp=1, pot=chem_pot, alpha=alpha, slab=slab, calc=lammps_calc, surface_name=surface_name, element=element, canonical=True, num_ads_atoms=num_ads_atoms, relax=True, ads_coords=select_positions)

/home/dux/miniconda3/envs/mlenv/lib/python3.8/site-packages/catkit/gen/surface.py:334: UserWarning: Your slab has no bulk atoms and may be too thin to identify surface atoms correctly. This may cause surface adsorption site identification to fail.
  warnings.warn(
INFO:Running with num_sweeps = 1, temp = 1, pot = 5, alpha = 0.99
INFO:there are 36 atoms 
INFO:using slab calc None
/home/dux/miniconda3/envs/mlenv/lib/python3.8/site-packages/catkit/gen/adsorption.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r1_topology = np.array(self.r1_topology)
/home/dux/miniconda3/envs/mlenv/lib/python3.8/site-packages/catkit/gen/adsorption.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or nd

LAMMPS (10 Feb 2021)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0.0000000 0.0000000 0.0000000) to (9.6488702 8.3561667 23.265754) with tilt (4.8244351 0.0000000 0.0000000)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  36 atoms
  read_data CPU = 0.000 seconds
36 atoms in group bulk
Reading tersoff potential file /home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_3x3/runs1_temp1_adsatoms95_alpha0.99_20220518-142257/GaN.tersoff with DATE: 2007-10-22
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 5.1
  ghost atom cutoff = 5.1
  binsize = 2.55, bins = 6 4 10
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair tersoff, perpetual
  

RuntimeError: Atoms object has no calculator.

In [ ]:
runs = range(1, num_runs+1)

# do the plots
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
ax[0, 0].plot(runs, energy_hist)
ax[0, 0].set_xlabel("Iter")
ax[0, 0].set_ylabel("Energy (E)")
ax[0, 0].set_title("Energy (E) vs Sweeps")

ax[0, 1].plot(runs, frac_accept_hist)
ax[0, 1].set_xlabel("Iter")
ax[0, 1].set_ylabel("Fraction accepted")
ax[0, 1].set_title("Fraction accepted vs Sweeps")

ax[1, 1].plot(runs, np.array(list(adsorption_count_hist.values())).T)
ax[1, 1].set_xlabel("Iter")
ax[1, 1].set_ylabel("Adsorption count")
ax[1, 1].legend(adsorption_count_hist.keys())
ax[1, 1].set_title("Adsorption count vs Iterations")

fig.show()
fig.tight_layout()